In [ ]:
# Setup: clone repo and install requirements
%cd /content  # go back to root
!rm -rf mixture-of-experts-project
!git clone https://github.com/moe-project-uu/mixture-of-experts-project.git || true
%cd mixture-of-experts-project
%pip install -r requirements.txt

In [ ]:
# run training script
!python scripts/train_cifar10.py

In [ ]:
# Load saved metrics for plotting
import torch
metrics = torch.load("checkpoints/resnet18_metrics.pt", map_location="cpu")
train_losses = metrics["train_losses"]
train_accs   = metrics["train_accs"]
val_losses   = metrics["val_losses"]
val_accs     = metrics["val_accs"]
len(train_losses), len(val_losses)

In [ ]:
# Plot curves
import matplotlib.pyplot as plt
plt.figure()
plt.plot(train_losses, label="train loss")
plt.plot(val_losses, label="val loss")
plt.xlabel("epoch"); plt.ylabel("loss"); plt.legend(); plt.show()

plt.figure()
plt.plot([x*100 for x in train_accs], label="train acc %")
plt.plot([x*100 for x in val_accs], label="val acc %")
plt.xlabel("epoch"); plt.ylabel("accuracy %"); plt.legend(); plt.show()